In [ ]:
# Guarded Setup
DRY_RUN = True
from notebooks._utils.common import *
CLI_OK = shell_available('forensic-cli')
LAB_ID = '00_introduction_and_setup'
LAB_ROOT = lab_root(LAB_ID)
print(f'CLI available: {CLI_OK}')
print(f'Artifacts root: {LAB_ROOT}')


# Lab 00 · Introduction and Guarded Setup


This notebook is part of the guarded, dry-run-first forensic lab suite.


## Objectives
- Understand the guardrails and dry-run defaults.
- Verify optional CLI availability.
- Bootstrap the Forensic Framework SDK in a lab workspace.


## Dry-Run Policy
All notebooks default to dry-run mode. CLI actions are only executed if `forensic-cli` is available.


In [ ]:
if CLI_OK:
    diag = run_cli(['forensic-cli', 'diagnostics', '--summary', '--dry-run'])
    print(diag.stdout.strip() or diag.stderr or f'Exit code: {diag.returncode}')
else:
    print('forensic-cli not detected; continuing with SDK-only workflow.')


## SDK Bootstrap


In [ ]:
from pathlib import Path
from forensic import ForensicFramework, __version__ as forensic_version
from forensic.core.config import get_config
workspace = LAB_ROOT / 'workspace'
framework = ForensicFramework(workspace=workspace)
version_snapshot = {
    'forensic_package_version': forensic_version,
    'workspace': str(framework.workspace),
    'config_timezone': framework.config.get('timezone'),
    'output_formats': framework.config.get('output_formats'),
}
output_path = workspace / f'{_ts()}_version_info.json'
json_dump_sorted(version_snapshot, output_path)
version_snapshot


### Configuration Precedence


In [ ]:
from pprint import pprint
defaults = get_config().as_dict()
yaml_cfg = get_config(config_root=CONFIG_ROOT).as_dict()
cli_override = get_config(config_root=CONFIG_ROOT, overrides={'log_level': 'DEBUG'}).as_dict()
precedence = {
    'defaults': defaults['log_level'],
    'yaml': yaml_cfg['log_level'],
    'cli_override': cli_override['log_level'],
}
json_dump_sorted(precedence, workspace / f'{_ts()}_config_precedence.json')
pprint(precedence)


### Checkpoint
This notebook is ready when the SDK workspace exists and the version snapshot JSON has been written.


In [ ]:
assert workspace.exists(), 'Workspace directory should be created.'
assert any(workspace.glob('*_version_info.json')), 'Version info JSON missing.'
